In [6]:
#populate polytopes 

from itertools import combinations
from sage.all import *

def uniform_corank_polytopes_from_subsets(n):
    """
    Generates a collection of polytopes P_S for all S subseteq [n].

    The polytope P_S is defined as the convex hull of the vertices:
    {1/n * (e_1 + ... + e_n)} U {e_i | i in S}, where e_i are standard basis vectors.
    """
    polytopes = {}
    
    # Define the first vertex, the centroid of the standard simplex
    centroid_vertex = vector(QQ, [1/n] * n)
    
    # Define the standard basis vectors
    basis_vectors = [vector(QQ, e) for e in identity_matrix(n).rows()]
    
    # Iterate through all possible subset sizes, from 0 to n
    for k in range(n + 1):
        # Use itertools.combinations to generate all subsets S of size k
        for s_tuple in combinations(range(n), k):
            # The indices for the subsets are 0-indexed, so we add 1 for the set [n]
            S = [i + 1 for i in s_tuple]
            
            # Construct the vertex list for the current polytope P_S
            vertices = [centroid_vertex]
            for i in S:
                vertices.append(basis_vectors[i - 1])
            
            # Create the Polyhedron object and store it
            # The vertices are implicitly cast to QQ, as `centroid_vertex` is already over QQ
            polytopes[tuple(S)] = Polyhedron(vertices=vertices)
            
    return polytopes

In [7]:
#Compute mixed volumes for these polytopes
def compute_mixed_volume(subset_tuple,d):
    """
    Computes the mixed volume V(P_{S_1}, ..., P_{S_d}) for d polytopes associated to U_{d,d+1}
    d = dimension
    """
    #subset_tuple should be length d
    if len(subset_tuple)!=d:
        print("tuple is wrong length")
    else:
        polytopes = uniform_corank_polytopes_from_subsets(d+1)
        # Define the standard basis vectors in R^{d+1} to cook up normalization
        basis_vectors = [vector(QQ, e) for e in identity_matrix(d+1).rows()]
        smallsimplexvertices = basis_vectors[:-1]
        center = 1/(d+1)*sum(basis_vectors)
        smallsimplexvertices.append(center)
        Q = Polyhedron(vertices=smallsimplexvertices)
        
        #The rest of the mixed volume computation
        unnormalized_mixed_volume = 0
        all_indices = range(d)
        
        #Range over size of subsets
        for i in range(1,d + 1):
            
            #Range over subsets of size i
            for subset_indices in combinations(all_indices, i):
                ###test
                #print("-"*40)
                #print(f"The subset(s) are:{tuple(subset_tuple[i] for i in subset_indices)}")
                ##
                #initialize current Minkowski sum
                origin = vector(QQ, [0]*(d+1))
                current_sum = Polyhedron(vertices=[origin])

                #initalize support
                support =()
                
                #Compute the support
                for index in subset_indices:
                    indexed_subset = subset_tuple[index]
                    support = tuple(set(support+indexed_subset))
                    
                    #also can compute the minkowksi sum over the subset indices here
                    current_sum += polytopes[indexed_subset]
                ###test
                #print(f"The current support is: {support}")
                #print(f"The current Minkowski sum is:{current_sum}")
                ##
                #If the support is too small, the volume is zero    
                if len(support)<d:
                    ##test
                    #print("It has trivial volume, since the support is not large enough")
                    continue
                #If the support is large enough, the we can work in the affine subspace    
                else:
                    ###test
                    #print(f"It has volume: {current_sum.volume(measure="induced")/Q.volume(measure="induced")}")
                    unnormalized_mixed_volume += (-1)**(d+i)*current_sum.volume(measure="induced")
                    #print(f"The updated volume is: {1/factorial(d)*unnormalized_mixed_volume/Q.volume(measure="induced")}")
        mixed_volume = 1/factorial(d)*unnormalized_mixed_volume/Q.volume(measure="induced") 
        #print(f"MV({subset_tuple})={mixed_volume}")
        return mixed_volume

In [8]:
##Compute transversals 
import collections
from itertools import product, permutations

def count_unordered_transversals(subsets):
    """
    Counts the number of unordered transversals for a given collection of subsets.

    Args:
        subsets: A list of lists or tuples, where each inner list/tuple is a subset.

    Returns:
        The number of unique unordered transversals.
    """
    n = len(subsets)
    if n == 0:
        return 1 # There is one empty transversal for an empty collection of sets.

    # Find the union of all subsets to determine the full range of elements
    all_elements = set(elem for s in subsets for elem in s)
    m = len(all_elements)

    # Sort the unique elements to have a consistent order for processing
    sorted_elements = sorted(list(all_elements))

    #generate ordered transversals, then put into a set
    ordered_transversals = set()
    for t in product(*subsets):
        # Check if the potential transversal contains distinct elements
        if len(set(t)) == n:
            # If it does, we have found an ordered transversal.
            # Convert to a frozenset to make it hashable and add to our set of unique transversals.
            ordered_transversals.add(frozenset(t))

    return len(ordered_transversals)


In [11]:
#reduce cases by symmetry classes
from itertools import product, combinations

def powerset(elements, include_empty=True):
    """Generate all subsets of elements"""
    s = list(elements)
    if include_empty:
        return [set(comb) for r in range(len(s)+1) for comb in combinations(s, r)]
    else:
        return [set(comb) for r in range(1, len(s)+1) for comb in combinations(s, r)]

def get_intersection_pattern(collection):
    """
    Get a canonical pattern based on subset sizes and intersection sizes.
    This ignores the specific element labels.
    """
    d = len(collection)
    
    # Get subset sizes sorted
    subset_sizes = tuple(sorted(len(subset) for subset in collection))
    
    # Get all pairwise intersection sizes as a sorted tuple
    pairwise_intersections = []
    for i in range(d):
        for j in range(i + 1, d):
            inter_size = len(collection[i] & collection[j])
            pairwise_intersections.append(inter_size)
    
    # Sort the pairwise intersections to make pattern order-independent
    pairwise_intersections.sort()
    
    # Create the pattern: (subset_sizes, pairwise_intersections)
    pattern = (subset_sizes, tuple(pairwise_intersections))
    
    return pattern

def get_symmetry_class_representatives(d, include_empty=False, min_union_size=None):
    """Generate one representative from each symmetry class based on intersection patterns"""
    universe = set(range(1, d+2))
    all_subsets = powerset(universe, include_empty=include_empty)
    
    seen_patterns = set()
    representatives = []
    
    for collection in product(all_subsets, repeat=d):
        # Apply filters
        if min_union_size is not None:
            union_set = set()
            for subset in collection:
                union_set |= subset
            if len(union_set) < min_union_size:
                continue
        
        # Get intersection pattern
        pattern = get_intersection_pattern(collection)
        
        if pattern not in seen_patterns:
            seen_patterns.add(pattern)
            representatives.append(collection)
    
    return representatives


In [15]:
#testing corank 1 uniform = volume polynomial
def test_corank_conjecture(d):
    representatives = get_symmetry_class_representatives(d, include_empty=False, min_union_size=d)
    for i, coll in enumerate(representatives):
        tup_coll = tuple(tuple(s) for s in coll)
        print('-'*40)
        #print(f"{coll}")
        #print(f"MV={int(compute_mixed_volume(tup_coll, d).n())}")
        #print(f"Unordered transversals count: {count_unordered_transversals(coll)}")
        if int(compute_mixed_volume(tup_coll, d).n()) != int(count_unordered_transversals(coll)):
            print(f"error at {coll}")
        else:
            continue
    print("process complete")   

In [17]:
#Example
test_corank_conjecture(4)

----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
----------------

process complete


In [18]:
#Fast(er) version of the conjecture testing
from itertools import product, combinations
from collections import Counter

def powerset_fast(elements, include_empty=True):
    """Generate all subsets of elements as tuples"""
    s = list(elements)
    if include_empty:
        return [tuple(sorted(comb)) for r in range(len(s)+1) for comb in combinations(s, r)]
    else:
        return [tuple(sorted(comb)) for r in range(1, len(s)+1) for comb in combinations(s, r)]

def get_intersection_pattern_fast(collection):
    """Get canonical intersection pattern"""
    d = len(collection)
    # Convert tuples to sets for intersection calculations
    collection_sets = [set(subset) for subset in collection]
    
    subset_sizes = tuple(sorted(len(subset) for subset in collection_sets))
    
    pairwise_intersections = []
    for i in range(d):
        for j in range(i + 1, d):
            inter_size = len(collection_sets[i] & collection_sets[j])
            pairwise_intersections.append(inter_size)
    
    pairwise_intersections.sort()
    return (subset_sizes, tuple(pairwise_intersections))

def is_valid_collection_fast(collection, min_union_size):
    """Check if collection meets all constraints"""
    # Check union size - convert tuples to sets for union
    union_set = set()
    for subset in collection:
        union_set |= set(subset)
    if len(union_set) < min_union_size:
        return False
    
    # Check excess repetition (tuples are hashable, so we can count directly)
    subset_counts = Counter(collection)
    for subset, count in subset_counts.items():
        if count > len(subset):
            return False
    
    return True

def get_symmetry_class_representatives_fast(d, include_empty=False, min_union_size=None):
    """Generate one representative from each symmetry class based on intersection patterns"""
    universe = tuple(range(1, d+2))
    all_subsets = powerset_fast(universe, include_empty=include_empty)
    
    seen_patterns = set()
    representatives = []
    
    for collection in product(all_subsets, repeat=d):
        # Apply filters
        if not is_valid_collection_fast(collection, min_union_size):
            continue
        
        # Get intersection pattern
        pattern = get_intersection_pattern_fast(collection)
        
        if pattern not in seen_patterns:
            seen_patterns.add(pattern)
            representatives.append(collection)
    
    return representatives

# Test function
def test_corank_conjecture_fast(d):
    representatives = get_symmetry_class_representatives_fast(d, include_empty=False, min_union_size=d)
    print(f"Testing d={d}, found {len(representatives)} symmetry classes")
    
    for i, coll in enumerate(representatives):
        tup_coll = coll  # Already tuples of tuples
        mv_value = compute_mixed_volume(tup_coll, d).n()
        trans_value = count_unordered_transversals(coll)
        
        mv_int = int(round(mv_value))
        
        if mv_int != trans_value:
            print(f"COUNTEREXAMPLE: {coll}")
            print(f"  Mixed Volume: {mv_int}, Transversals: {trans_value}")
            return False
    
    print(f"Conjecture holds for d={d}!")
    return True

Testing d=4, found 523 symmetry classes
Conjecture holds for d=4!
Testing d=5, found 8035 symmetry classes
Conjecture holds for d=5!


In [ ]:
#example
test_corank_conjecture_fast(6)